# Modelado y Validación del Modelo de Regresión

En este notebook se desarrollan los modelos de regresión para predecir la variable objetivo del dataset proporcionado por el curso.

El objetivo es:
- Entrenar distintos modelos de regresión.
- Evaluar su rendimiento mediante métricas estándar (RMSE, MAE, R²).
- Controlar problemas de overfitting y underfitting.
- Seleccionar y guardar el mejor modelo para su uso en la aplicación interactiva.


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import joblib


In [2]:
# Carga del dataset procesado
df = pd.read_csv("train_ready_for_modeling.csv")

df.head()



,brand,model,model_year,milage,engine,transmission,ext_col,int_col,price,fuel_type_E85 Flex Fuel,fuel_type_Gasoline,fuel_type_Hybrid,fuel_type_Plug-In Hybrid,fuel_type_Unknown,fuel_type_not supported,fuel_type_–,accident_None reported,clean_title_Yes
0,MINI,Cooper S Base,2007,213000,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,4200,False,True,False,False,False,False,False,True,True
1,Lincoln,LS V8,2002,143250,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,4999,False,True,False,False,False,False,False,False,True
2,Chevrolet,Silverado 2500 LT,2002,136731,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,13900,True,False,False,False,False,False,False,True,True
3,Genesis,G90 5.0 Ultimate,2017,19500,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,45000,False,True,False,False,False,False,False,True,True
4,Mercedes-Benz,Metris Base,2021,7388,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,97500,False,True,False,False,False,False,False,True,True


#Evaluación de modelos

Para poder comparar de forma consistente los distintos modelos de regresión entrenados, se define una función auxiliar que centraliza el proceso de entrenamiento y evaluación.

Esta función entrena el modelo con el conjunto de entrenamiento y calcula métricas estándar de regresión (RMSE, MAE y R²) tanto en entrenamiento como en test. De esta forma, se puede analizar el rendimiento predictivo de cada modelo y detectar posibles problemas de overfitting o underfitting comparando ambos resultados.

In [3]:
def evaluate_model(model, X_train, y_train, X_test, y_test):

    # Entrenamiento
    model.fit(X_train, y_train)

    # Predicciones
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Métricas
    metrics = {
        "train_rmse": np.sqrt(mean_squared_error(y_train, y_train_pred)),
        "test_rmse": np.sqrt(mean_squared_error(y_test, y_test_pred)),
        "mae": mean_absolute_error(y_test, y_test_pred),
        "r2": r2_score(y_test, y_test_pred)
    }

    return metrics


## División de datos en Training y Test

El dataset se divide en conjuntos de entrenamiento y prueba con el objetivo de evaluar la capacidad de generalización de los modelos.  
Se utiliza una división 80/20 y una semilla fija para garantizar reproducibilidad.


In [4]:
# Definición de variables predictoras y variable objetivo
X = df.drop("price", axis=1)
y = df["price"]

# Codificación de variables categóricas
X_encoded = pd.get_dummies(X, drop_first=True)  # drop_first=True para evitar multicolinealidad

# División original 80/20 (se usará más adelante para modelos complejos)
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(150826, 3603) (37707, 3603) (150826,) (37707,)


## Modelo Base: Regresión Lineal

Se utiliza un modelo de Regresión Lineal como baseline para establecer un punto de referencia.  
Este modelo permite evaluar si modelos más complejos aportan una mejora significativa en el rendimiento.


In [5]:
# Para que funcione rápido, usamos solo una muestra pequeña del dataset
X_sample, _, y_sample, _ = train_test_split(X_encoded, y, test_size=0.7, random_state=42)
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)
# LinearRegression sobre la muestra
lr = LinearRegression()
lr_metrics = evaluate_model(lr, X_train_s, y_train_s, X_test_s, y_test_s)

print("Linear Regression metrics (sampled data):")
print(lr_metrics)

Linear Regression metrics (sampled data):
{'train_rmse': np.float64(70865.96542614607), 'test_rmse': np.float64(90017.97260528362), 'mae': 24491.809071559175, 'r2': 0.05815922485658642}


#Conclusión
El modelo de regresión lineal funciona como baseline, ofreciendo un r² bajo (≈0.06) y errores relativamente altos, lo que indica que la relación entre las variables y el precio de los coches es compleja y difícil de capturar con un modelo lineal simple. Esto nos muestra la necesidad de explorar modelos más avanzados para mejorar la capacidad predictiva.

## Modelo Ensemble: Random Forest Regressor

Random Forest es un modelo ensemble basado en múltiples árboles de decisión.  
Permite capturar relaciones no lineales y reducir la varianza respecto a modelos individuales.


In [7]:
#Random Forest Regressor

#Tomamos muestra del dataset

X_rf_sample, _, y_rf_sample, _ = train_test_split(
    X_encoded, y, test_size=0.7, random_state=42
)

X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(
    X_rf_sample, y_rf_sample, test_size=0.2, random_state=42
)

rf = RandomForestRegressor(
    n_estimators=30,
    max_depth=12,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

rf_metrics = evaluate_model(rf, X_train_rf, y_train_rf, X_test_rf, y_test_rf)

print("Random Forest metrics (sampled data):")
rf_metrics


Random Forest metrics (sampled data):


{'train_rmse': np.float64(69572.35758480337),
 'test_rmse': np.float64(88141.08477705921),
 'mae': 21365.507594375224,
 'r2': 0.09702481420152254}

## Modelo Ensemble: Gradient Boosting Regressor

Gradient Boosting construye modelos secuenciales que corrigen los errores del modelo anterior.  
Suele ofrecer un buen equilibrio entre sesgo y varianza en problemas de regresión.


In [8]:
# Tomamos una muestra razonable para Gradient Boosting
X_gb_sample, _, y_gb_sample, _ = train_test_split(
    X_encoded, y, test_size=0.7, random_state=42
)
X_train_gb, X_test_gb, y_train_gb, y_test_gb = train_test_split(
    X_gb_sample, y_gb_sample, test_size=0.2, random_state=42
)

from sklearn.ensemble import GradientBoostingRegressor

# Gradient Boosting sobre la muestra
gb = GradientBoostingRegressor(
    n_estimators=50,       # menos árboles para que sea rápido
    max_depth=3,           # profundidad moderada
    learning_rate=0.1,
    random_state=42
)

gb_metrics = evaluate_model(gb, X_train_gb, y_train_gb, X_test_gb, y_test_gb)

print("Gradient Boosting metrics (sampled data):")
print(gb_metrics)


Gradient Boosting metrics (sampled data):
{'train_rmse': np.float64(67604.33757196736), 'test_rmse': np.float64(88822.040128608), 'mae': 21619.831700237264, 'r2': 0.08301861013989698}


#Modelo Ensemble: XGBoost Regressor
XGBoost es un modelo de gradient boosting optimizado, que construye árboles secuenciales corrigiendo errores del anterior, pero con mejoras de velocidad y regularización.
Suele ofrecer un buen equilibrio entre sesgo y varianza y manejar datasets grandes de manera eficiente

In [9]:
# Importamos el modelo
from xgboost import XGBRegressor

# Tomamos la misma muestra que Random Forest y Gradient Boosting
X_xgb_sample, _, y_xgb_sample, _ = train_test_split(
    X_encoded, y, test_size=0.7, random_state=42
)
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(
    X_xgb_sample, y_xgb_sample, test_size=0.2, random_state=42
)

# Definimos el modelo XGBoost
xgb_model = XGBRegressor(
    n_estimators=50,   # número de árboles
    max_depth=3,       # profundidad de cada árbol
    learning_rate=0.1, # cada árbol corrige solo parcialmente el error del anterior
    random_state=42,
    n_jobs=-1
)

# Evaluamos el modelo usando la misma función que tus otros modelos
xgb_metrics = evaluate_model(xgb_model, X_train_xgb, y_train_xgb, X_test_xgb, y_test_xgb)

print("XGBoost metrics (sampled data):")
print(xgb_metrics)


XGBoost metrics (sampled data):
{'train_rmse': np.float64(69755.61821100864), 'test_rmse': np.float64(88343.45195881808), 'mae': 21569.115234375, 'r2': 0.09287363290786743}


## Comparación de modelos

Se comparan los distintos modelos entrenados utilizando métricas de regresión (RMSE, MAE y R²)  
con el objetivo de seleccionar el modelo con mejor rendimiento y menor sobreajuste.


In [11]:
# Comparación de métricas entre modelos
results = pd.DataFrame([
     ["Linear Regression", lr_metrics["test_rmse"], lr_metrics["mae"], lr_metrics["r2"]],
     ["Random Forest", rf_metrics["test_rmse"], rf_metrics["mae"], rf_metrics["r2"]],
     ["Gradient Boosting", gb_metrics["test_rmse"], gb_metrics["mae"], gb_metrics["r2"]],
     ["XGBoost", xgb_metrics["test_rmse"], xgb_metrics["mae"], xgb_metrics["r2"]]
 ], columns=["Model", "RMSE", "MAE", "R2"])


In [12]:
print(results)

               Model          RMSE           MAE        R2
0  Linear Regression  90017.972605  24491.809072  0.058159
1      Random Forest  88141.084777  21365.507594  0.097025
2  Gradient Boosting  88822.040129  21619.831700  0.083019
3            XGBoost  88343.451959  21569.115234  0.092874


#Conclusión:
En esta muestra, Random Forest parece ser el modelo con mejor desempeño, aunque todavía no es espectacular. Esto es normal porque estamos usando solo una muestra pequeña del dataset para poder entrenar, ya que el dataset es demasiado grande.

Linear Regression: Rendimiento muy bajo (R² ≈ 0.058), lo que indica que el modelo captura solo una pequeña parte de la relación entre las variables y el precio.

Random Forest: Mejora bastante respecto a la regresión lineal, R² positivo aunque bajo (0.097).

Gradient Boosting: Muy parecido a Random Forest, ligeramente peor en R² y RMSE.

XGBoost: Rendimiento muy similar a Random Forest (R² 0.093), con un RMSE ligeramente superior. Es eficiente y rápido, pero en esta muestra no supera al Random Forest.

## Validación cruzada (muestra del dataset)

Se aplica validación cruzada (K-Fold) sobre una muestra del dataset para evaluar la estabilidad del modelo seleccionado y detectar posibles problemas de overfitting o underfitting, sin que el entrenamiento se vuelva computacionalmente costoso.

In [13]:
from sklearn.model_selection import cross_val_score

# # Validación cruzada K-Fold con el Random Forest de la muestra (n_estimators=30, max_depth=12, min_samples_leaf=5)
rf = RandomForestRegressor(
    n_estimators=30,
    max_depth=12,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

# Validación cruzada K-Fold con 5 splits
cv_scores = cross_val_score(rf, X_train_rf, y_train_rf, cv=5, scoring='r2', n_jobs=-1)

print("R2 scores for each fold:", cv_scores)
print("Mean R2:", cv_scores.mean())
print("Standard deviation:", cv_scores.std())



R2 scores for each fold: [0.0930626  0.11005882 0.08352433 0.15097889 0.07922649]
Mean R2: 0.10337022552168562
Standard deviation: 0.026055130970248728


#Conclusión de la validación cruzada

El modelo Random Forest entrenado sobre la muestra muestra un R² promedio de ~0.10, lo que indica que explica aproximadamente un 10 % de la variabilidad de los precios en esta muestra.

Los valores de R² por fold oscilan entre 0.08 y 0.15, lo que indica cierta variabilidad entre los subsets, pero no hay splits que fallen completamente (no hay valores negativos).

La desviación estándar (~0.026) es baja, lo que sugiere que el modelo es relativamente estable entre los distintos folds, aunque su poder predictivo aún es limitado.

En conjunto, esto indica que el modelo generaliza de manera consistente sobre la muestra, pero todavía no captura toda la complejidad del dataset completo.

## Optimización de hiperparámetros

Se realiza una búsqueda de hiperparámetros para mejorar el rendimiento del modelo  
y reducir posibles problemas de sobreajuste.


In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


X_gs_sample, _, y_gs_sample, _ = train_test_split(X_encoded, y, test_size=0.7, random_state=42)
X_train_gs, X_test_gs, y_train_gs, y_test_gs = train_test_split(X_gs_sample, y_gs_sample, test_size=0.2, random_state=42)

# Modelo base
rf = RandomForestRegressor(random_state=42, n_jobs=-1)

# Hiperparámetros (mínimos para rapidez)
param_grid = {
    "n_estimators": [10, 20],
    "max_depth": [6, 8],
    "min_samples_leaf": [3, 5]
}

# GridSearchCV rápido
grid_search = GridSearchCV(rf, param_grid, cv=3, scoring='r2', n_jobs=-1)
grid_search.fit(X_train_gs, y_train_gs)

print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor R2 (media CV):", grid_search.best_score_)



Mejores hiperparámetros: {'max_depth': 8, 'min_samples_leaf': 5, 'n_estimators': 20}
Mejor R2 (media CV): 0.09088790354547838


#Conclusión de la optimización de hiperparámetros:
La búsqueda de hiperparámetros sobre la muestra del dataset ha mostrado que la combinación óptima para el Random Forest es: max_depth=8, min_samples_leaf=5 y n_estimators=20. Con estos parámetros, el modelo presenta un R² medio de aproximadamente 0.091 en validación cruzada, lo que indica que, aunque hemos logrado reducir algo la complejidad y evitar sobreajuste, el modelo aún explica una pequeña parte de la varianza de la variable objetivo en la muestra utilizada. Esto confirma que el problema de predicción del precio es complejo y que la relación entre las variables es relativamente difícil de capturar con un modelo de Random Forest simple sobre esta muestra.

## Guardado del modelo final

El modelo seleccionado se guarda para su posterior uso en la aplicación interactiva  
desarrollada con Streamlit.


In [16]:
import joblib

# Entrenamos el modelo final con los mejores hiperparámetros sobre la muestra
best_rf = RandomForestRegressor(
    n_estimators=20,
    max_depth=8,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1
)

best_rf.fit(X_train_rf, y_train_rf)

# Guardamos el modelo
joblib.dump(best_rf, "best_rf_model_sample.pkl")

print("Modelo guardado como 'best_rf_model_sample.pkl'")



Modelo guardado como 'best_rf_model_sample.pkl'


El modelo Random Forest con los mejores hiperparámetros (n_estimators=20, max_depth=8, min_samples_leaf=5) ha sido entrenado sobre la muestra y guardado para su uso en la aplicación interactiva. Esto permite cargar el modelo directamente sin necesidad de reentrenarlo cada vez, facilitando la predicción de precios de forma rápida.